## Notebook setup

In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import base64
import pandas as pd
from typing import List

from IPython.display import HTML

import openbb_terminal.config_terminal as cfg
from openbb_terminal.helper_classes import TerminalStyle
from openbb_terminal import OpenBBFigure, theme


warnings.filterwarnings("ignore")

pd.set_option("display.float_format", lambda x: "%.5f" % x)

In [ ]:
# Suppressing sdk logs
from openbb_terminal.core.session.current_system import set_system_variable

set_system_variable("LOGGING_SUPPRESS", True)

# Import the OpenBB SDK
from openbb_terminal.sdk import openbb, widgets

In [ ]:
from openbb_terminal.core.session.current_user import get_current_user

user = get_current_user()

cfg.theme = TerminalStyle(
    user.preferences.MPL_STYLE,
    user.preferences.PMF_STYLE,
    user.preferences.RICH_STYLE,
)
theme.apply_style("light")

In [ ]:
stylesheet = widgets.html_report_stylesheet()

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
# Do not leave parameters blank as notebook will not run otherwise
symbol = "BTC"
report_name = f"Crypto Report for {symbol}"

In [ ]:
ticker = symbol

In [ ]:
author = ""
report_title = f"INVESTMENT RESEARCH REPORT ON {ticker.upper()}"
report_date = datetime.datetime.now().strftime("%d %B, %Y")
report_time = datetime.datetime.now().strftime("%H:%M")
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

In [ ]:
# basic info

basic_info = openbb.crypto.dd.basic(ticker)
if not basic_info.empty:
    basic_info["Metric"] = basic_info["Metric"].str.title()
    basic_info["Metric"] = basic_info["Metric"].str.replace("_", " ")
    basic_info = basic_info.set_index("Metric")
    basic_info

In [ ]:
news = openbb.crypto.dd.news(symbol=ticker).head(10)

if (isinstance(news, pd.DataFrame) and not news.empty) or (
    not isinstance(news, pd.DataFrame) and news
):
    news = news.set_index("published_at")
    news.sort_index()
    news["link"] = news["link"].apply(lambda x: f'<a href="{x}">{x}</a>')
    news
else:
    news = pd.DataFrame()

In [ ]:
links = openbb.crypto.dd.links(ticker)

if (isinstance(links, pd.DataFrame) and not links.empty) or (
    not isinstance(links, pd.DataFrame) and links
):
    links["Link"] = links["Link"].apply(lambda x: f'<a href="{x}">{x}</a>')
    links
else:
    links = pd.DataFrame()

In [ ]:
five_year = datetime.datetime.now() - datetime.timedelta(days=1825)
three_year = datetime.datetime.now() - datetime.timedelta(days=1095)
one_year = datetime.datetime.now() - datetime.timedelta(days=365)
three_months = datetime.datetime.now() - datetime.timedelta(days=90)

dates = {
    "three_months": three_months,
    "one_year": one_year,
    "three_year": three_year,
    "five_year": five_year,
}

In [ ]:
from tkinter.filedialog import Open


def get_historical_data(ticker, dict_of_dates, is_plot=True) -> List[pd.DataFrame]:
    list_of_historical_data = []

    for date in dict_of_dates:
        data = openbb.crypto.load(
            symbol=ticker,
            to_symbol="usd",
            start_date=dict_of_dates[date],
            source="YahooFinance",
        )

        if data.empty:
            continue

        # we'll be using the adjusted close price as same as close price
        data["Adj Close"] = data["Close"]

        if is_plot:
            fig = OpenBBFigure(yaxis_title="USDT").set_title(
                f"{date} historical data for {ticker.upper()}"
            )
            fig.add_scatter(x=data.index, y=data["Adj Close"], name="Close")

            plot_data = fig.to_html()

            list_of_historical_data.append(plot_data)
        else:
            list_of_historical_data.append(data)

    return list_of_historical_data

In [ ]:
try:
    historical_data = get_historical_data(ticker, dates)
except Exception as e:
    raise KeyboardInterrupt

In [ ]:
historical_data_one_year = get_historical_data(ticker, {"one_year": one_year}, False)[0]
historical_data_one_year

In [ ]:
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import show_quick_performance

show_quick_performance(
    crypto_df=historical_data_one_year,
    symbol=ticker,
    current_currency="USDT",
    source="YahooFinance",
    exchange="binance",
    interval="1440",
)

In [ ]:
headlines = openbb.crypto.dd.headlines_chart(ticker, external_axes=True)

if headlines:
    headlines = headlines.to_html()

In [ ]:
candle_one_year = openbb.crypto.chart(
    historical_data_one_year,
    to_symbol=ticker,
    from_symbol="USDT",
    source="YahooFinance",
    exchange="binance",
    interval="1440",
    external_axes=True,
)

if candle_one_year:
    candle_one_year = candle_one_year.to_html()

## Due Diligence

In [ ]:
# market cap dominance
mcapdom_df = openbb.crypto.dd.mcapdom(symbol=ticker)

if (isinstance(mcapdom_df, pd.DataFrame) and not mcapdom_df.empty) or (
    not isinstance(mcapdom_df, pd.DataFrame) and mcapdom_df
):
    mcapdom = openbb.crypto.dd.mcapdom_chart(symbol=ticker, external_axes=True)

    if mcapdom:
        mcapdom = mcapdom.to_html()
else:
    mcapdom_df = pd.DataFrame()

In [ ]:
# roadmap
roadmap_model = openbb.crypto.dd.rm(symbol=ticker)

if (isinstance(roadmap_model, pd.DataFrame) and not roadmap_model.empty) or (
    not isinstance(roadmap_model, pd.DataFrame) and roadmap_model
):
    roadmap_model = roadmap_model.set_index("Date")
    roadmap_model.sort_index()

    roadmap_view = openbb.crypto.dd.rm_chart(symbol=ticker, external_axes=True)
    if roadmap_view:
        roadmap_view = roadmap_view.to_html()

else:
    roadmap_model = pd.DataFrame()

In [ ]:
tokenomics_model = openbb.crypto.dd.rm(symbol=ticker)

if (isinstance(tokenomics_model, pd.DataFrame) and not tokenomics_model.empty) or (
    not isinstance(tokenomics_model, pd.DataFrame) and tokenomics_model
):
    tokenomics_model = tokenomics_model.set_index("Date")
    tokenomics_model.sort_index()

    tokenomics_view = openbb.crypto.dd.tk_chart(symbol=ticker, external_axes=True)
    if tokenomics_view:
        tokenomics_view = tokenomics_view.to_html()

else:
    tokenomics_model = pd.DataFrame()

In [ ]:
# fundraise
fundraise = openbb.crypto.dd.fr(symbol=ticker)  # this returns a tuple

if fundraise:
    fundraise_info = fundraise[0]
    fundraise_df = fundraise[1]

In [ ]:
trading_pair_info = openbb.crypto.dd.trading_pair_info(f"{ticker}-USDT")
if (isinstance(trading_pair_info, pd.DataFrame) and not trading_pair_info.empty) or (
    not isinstance(trading_pair_info, pd.DataFrame) and trading_pair_info
):
    trading_pair_info["Metric"] = trading_pair_info["Metric"].str.title()
    trading_pair_info["Metric"] = trading_pair_info["Metric"].str.replace("_", " ")
    trading_pair_info = trading_pair_info.set_index("Metric")
    trading_pair_info
else:
    trading_pair_info = pd.DataFrame()

In [ ]:
import openbb_terminal.cryptocurrency.due_diligence.pycoingecko_model as gecko
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coingecko_id

ath = pd.DataFrame()
try:
    cg_id = get_coingecko_id(symbol=ticker)
    coin = gecko.Coin(ticker)
    ath = coin.get_all_time_high(currency=cg_id)

except Exception as e:
    print("Reached coingecko API limit")

In [ ]:
import openbb_terminal.cryptocurrency.due_diligence.pycoingecko_model as gecko
from openbb_terminal.cryptocurrency.cryptocurrency_helpers import get_coingecko_id

atl = pd.DataFrame()
try:
    cg_id = get_coingecko_id(symbol=ticker)
    coin = gecko.Coin(ticker)
    atl = coin.get_all_time_low(currency=cg_id)

except Exception as e:
    print("Reached coingecko API limit")

In [ ]:
# market stats about loaded coin

market = openbb.crypto.dd.mkt(ticker)

if (isinstance(market, pd.DataFrame) and not market.empty) or (
    not isinstance(market, pd.DataFrame) and market
):
    market = market.set_index("exchange")

    market["market_url"] = market["market_url"].apply(
        lambda x: f'<a href="{x}">{x}</a>'
    )
    market
else:
    market = pd.DataFrame()

In [ ]:
# all exchanges where loaded coin is listed
ex = openbb.crypto.dd.ex(ticker)

if (isinstance(ex, pd.DataFrame) and not ex.empty) or (
    not isinstance(ex, pd.DataFrame) and ex
):
    ex = ex.set_index("id")
    ex
else:
    ex = pd.DataFrame()

In [ ]:
# last trades
trades = openbb.crypto.dd.trades(symbol=ticker, exchange_id="gemini", to_symbol="USD")
if (isinstance(trades, pd.DataFrame) and not trades.empty) or (
    not isinstance(trades, pd.DataFrame) and trades
):
    trades = trades.set_index("Date")
    trades
else:
    trades = pd.DataFrame()

In [ ]:
# order book
order_book_raw = openbb.crypto.dd.ob(
    symbol=ticker, exchange="gemini", to_symbol="USD"
)  # this returns a dict

if order_book_raw:
    order_book = openbb.crypto.dd.ob_chart(
        symbol=ticker, exchange="gemini", to_symbol="USD", external_axes=True
    )
    if order_book:
        order_book = order_book.to_html()

In [ ]:
# price and supply related metrics for loaded coin

ps = openbb.crypto.dd.ps(ticker)
if (isinstance(ps, pd.DataFrame) and not ps.empty) or (
    not isinstance(ps, pd.DataFrame) and ps
):
    ps = ps.set_index("Metric")
    ps
else:
    ps = pd.DataFrame()

In [ ]:
# events

events = openbb.crypto.dd.events(ticker)

if (isinstance(events, pd.DataFrame) and not events.empty) or (
    not isinstance(events, pd.DataFrame) and events
):
    events = events.set_index("date")
    events["link"] = events["link"].apply(lambda x: f'<a href="{x}">{x}</a>')
else:
    events = pd.DataFrame()

In [ ]:
# social media

from openbb_terminal.cryptocurrency.due_diligence.pycoingecko_view import display_social

display_social(ticker)

In [ ]:
# tweets for loaded coin

tweets = openbb.crypto.dd.twitter(ticker)
tweets

In [ ]:
# github activity over time

gh_data = openbb.crypto.dd.gh(symbol=ticker)

if (isinstance(gh_data, pd.DataFrame) and not gh_data.empty) or (
    not isinstance(gh_data, pd.DataFrame) and gh_data
):
    gh = openbb.crypto.dd.gh_chart(symbol=ticker, external_axes=True)
    if gh:
        gh = gh.to_html()
else:
    gh_data = pd.DataFrame()

## TA


In [ ]:
ticker_data = historical_data_one_year.copy()

In [ ]:
# simple moving average
ticker_data.index.names = ["date"]

sma = openbb.ta.ma_chart(
    data=ticker_data,
    ma_type="SMA",
    symbol=ticker,
    external_axes=True,
)

if sma:
    sma = sma.to_html()

In [ ]:
# exponential moving average
ema = openbb.ta.ma_chart(
    data=ticker_data,
    ma_type="EMA",
    symbol=ticker,
    external_axes=True,
)

if ema:
    ema = ema.to_html()

In [ ]:
# zero lag exponential moving average

zlma = openbb.ta.ma_chart(
    data=ticker_data,
    ma_type="ZLMA",
    symbol=ticker,
    external_axes=True,
)

if zlma:
    zlma = zlma.to_html()

In [ ]:
# commodity channel index

cci = openbb.ta.cci_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=True,
)

if cci:
    cci = cci.to_html()

In [ ]:
# moving average convergence/divergence

macd = openbb.ta.macd_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=True,
)

if macd:
    macd = macd.to_html()

In [ ]:
# fisher transform

fisher = openbb.ta.fisher_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=True,
)

if fisher:
    fisher = fisher.to_html()

In [ ]:
# aroon indicator

aroon = openbb.ta.aroon_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=True,
)

if aroon:
    aroon = aroon.to_html()

In [ ]:
# bollinger bands

bbands = openbb.ta.bbands_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=True,
)

if bbands:
    bbands = bbands.to_html()

In [ ]:
# fibonacci retracement

fib = openbb.ta.fib_chart(
    data=ticker_data,
    symbol=ticker,
    external_axes=True,
)

if fib:
    fib = fib.to_html()

## QA

In [ ]:
# summary

summary = openbb.qa.summary(data=ticker_data)
summary

In [ ]:
# normality statistics and tests

# openbb.qa.normality_chart(
#     data=ticker_data,
#     target="Close",
# )

normality = openbb.qa.normality(
    data=ticker_data["Close"],
)
normality

In [ ]:
# box and whisker plot

bw = openbb.qa.bw(
    symbol=ticker,
    data=ticker_data,
    target="Close",
    yearly=False,
    external_axes=True,
)

if bw:
    bw = bw.to_html()

In [ ]:
# rolling mean and std deviation of prices

rolling = openbb.qa.rolling_chart(
    symbol=ticker,
    data=ticker_data,
    target="Close",
    external_axes=True,
)

if rolling:
    rolling.update_layout(
        legend=dict(
            x=0.01,
            y=0.99,
            xanchor="left",
            yanchor="top",
            xref="paper",
            yref="paper",
            bgcolor="rgba(0, 0, 0, 0)",
        ),
        legend2=dict(bgcolor="rgba(0, 0, 0, 0)"),
    )
    rolling = rolling.to_html()

In [ ]:
# rolling kurtosis of distribution of prices

kurtosis = openbb.qa.kurtosis_chart(
    symbol=ticker,
    data=ticker_data,
    target="Close",
    external_axes=True,
)

if kurtosis:
    kurtosis = kurtosis.to_html()

## Render the report template to a file

In [ ]:
body = ""


body += widgets.header(
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>INVESTMENT RESEARCH REPORT:</b> {ticker}",
    plotly_js=True,
)

body += widgets.tablinks(
    [
        "SUMMARY",
        "Due Diligence",
        "Technical Analysis",
        "Quantitative Analysis",
    ]
)

comment = ""

# Summary tab
htmlcode = ""
htmlcode += widgets.row(
    [widgets.h(3, "Basic information about loaded coin") + basic_info.to_html()]
)
if not links.empty:
    htmlcode += widgets.row([widgets.h(3, "Links") + links.to_html(escape=False)])

if not news.empty:
    htmlcode += widgets.row(
        [widgets.h(3, "Published news") + news.to_html(escape=False)]
    )

htmlcode += widgets.row([widgets.h(3, "Historical data")])
for data in historical_data:
    htmlcode += widgets.row([data])

htmlcode += widgets.row([widgets.h(3, "1-year historical data candlestick chart")])
htmlcode += widgets.row([candle_one_year])
htmlcode += widgets.row([widgets.h(3, "Sentiment analysis")])
htmlcode += widgets.row([headlines])
# TODO: print rich table
# htmlcode += widgets.row([widgets.h(3, "Quick performance")])
# htmlcode += widgets.row([quick_performance])
body += widgets.add_tab("SUMMARY", htmlcode)

# # Due diligence tab
htmlcode = ""
if not trading_pair_info.empty:
    htmlcode += widgets.row(
        [widgets.h(3, f"Trading pair info {ticker}/USDT") + trading_pair_info.to_html()]
    )
if not mcapdom_df.empty:
    htmlcode += widgets.row([widgets.h(3, "Market cap dominance")])
    htmlcode += widgets.row([mcapdom])

if not roadmap_model.empty:
    htmlcode += widgets.row([widgets.h(3, "Roadmap") + roadmap_model.to_html()])
    htmlcode += widgets.row([roadmap_view])

if not tokenomics_model.empty:
    htmlcode += widgets.row([widgets.h(3, "Tokenomics") + tokenomics_model.to_html()])
    htmlcode += widgets.row([tokenomics_view])

if fundraise:
    htmlcode += widgets.row([widgets.h(3, "Fundraising Details")])
    htmlcode += widgets.row([widgets.p(fundraise_info)])
    htmlcode += widgets.row([fundraise_df.to_html()])

if not ath.empty:
    htmlcode += widgets.row([widgets.h(3, "All time high")])
    htmlcode += widgets.row([ath.to_html()])

if not atl.empty:
    htmlcode += widgets.row([widgets.h(3, "All time low")])
    htmlcode += widgets.row([atl.to_html()])

if not market.empty:
    htmlcode += widgets.row([widgets.h(3, "All markets")])
    htmlcode += widgets.row([market.sample(n=20).sort_index().to_html(escape=False)])

if not ex.empty:
    htmlcode += widgets.row([widgets.h(3, "Exchanges where loaded coin is listed")])
    htmlcode += widgets.row([ex.sample(n=20).sort_index().to_html()])

if not trades.empty:
    htmlcode += widgets.row([widgets.h(3, "Last trades")])
    htmlcode += widgets.row([trades.tail(20).to_html()])

if order_book_raw:
    htmlcode += widgets.row([widgets.h(3, "Order book")])
    htmlcode += widgets.row([order_book])

if not ps.empty:
    htmlcode += widgets.row(
        [widgets.h(3, "Price and supply related metrics for loaded coin")]
    )
    htmlcode += widgets.row([ps.to_html()])

if not events.empty:
    htmlcode += widgets.row([widgets.h(3, "Events")])
    htmlcode += widgets.row([events.to_html(escape=False)])

# TODO: print rich table
# htmlcode += widgets.row([widgets.h(3, "Tweets for loaded coin")])
# htmlcode += widgets.row([tweets])
# htmlcode += widgets.row([widgets.h(3, "Social media")])
# htmlcode += widgets.row([social.to_html()])

if not gh_data.empty:
    htmlcode += widgets.row([widgets.h(3, "Github activity over time")])
    htmlcode += widgets.row([gh])
else:
    htmlcode += widgets.row([widgets.h(3, "Github activity over time")])
    htmlcode += widgets.row([widgets.p("No data available.")])

body += widgets.add_tab("Due Diligence", htmlcode)

# Technical Analysis tab
htmlcode = ""
htmlcode = widgets.row([widgets.h(3, "Simple moving average")])
htmlcode += widgets.row([sma])
htmlcode += widgets.row([widgets.h(3, "Exponential moving average")])
htmlcode += widgets.row([ema])
htmlcode += widgets.row([widgets.h(3, "Zero lag exponential moving average")])
htmlcode += widgets.row([zlma])
htmlcode += widgets.row([widgets.h(3, "Commodity channel index")])
htmlcode += widgets.row([cci])
htmlcode += widgets.row([widgets.h(3, "Moving average convergence/divergence")])
htmlcode += widgets.row([macd])
htmlcode += widgets.row([widgets.h(3, "Fisher transform")])
htmlcode += widgets.row([fisher])
htmlcode += widgets.row([widgets.h(3, "Aroon indicator")])
htmlcode += widgets.row([aroon])
htmlcode += widgets.row([widgets.h(3, "Bollinger bands")])
htmlcode += widgets.row([bbands])
htmlcode += widgets.row([widgets.h(3, "Fibonacci retracement")])
htmlcode += widgets.row([fib])
body += widgets.add_tab("Technical Analysis", htmlcode)

# Quantitative Analysis tab
htmlcode = ""
htmlcode = widgets.row([widgets.h(3, "Summary") + summary.to_html()])
htmlcode += widgets.row(
    [widgets.h(3, "Normality statistics and tests") + normality.to_html()]
)
htmlcode += widgets.row([widgets.h(3, "Box and whisker plot")])
htmlcode += widgets.row([bw])
htmlcode += widgets.row([widgets.h(3, "Rolling mean and std deviation of prices")])
htmlcode += widgets.row([rolling])
htmlcode += widgets.row([widgets.h(3, "Rolling kurtosis of distribution of prices")])
htmlcode += widgets.row([kurtosis])
body += widgets.add_tab("Quantitative Analysis", htmlcode)

body += widgets.tab_clickable_and_save_evt()
report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# # to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)